# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']


L. Boogaard  ->  L. Boogaard  |  ['L. Boogaard']
S. Bosman  ->  S. Bosman  |  ['S. Bosman']
L. Boogaard  ->  L. Boogaard  |  ['L. Boogaard']
F. Walter  ->  F. Walter  |  ['F. Walter']
Arxiv has 83 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2309.05714


extracting tarball to tmp_2309.05714...

 done.


list index out of range
Retrieving document from  https://arxiv.org/e-print/2309.06235


...

extracting tarball to tmp_2309.06311... done.


L. Boogaard  ->  L. Boogaard  |  ['L. Boogaard']


Found 52 bibliographic references in tmp_2309.06311/ms.bbl.
Retrieving document from  https://arxiv.org/e-print/2309.06319


extracting tarball to tmp_2309.06319... done.


Found 79 bibliographic references in tmp_2309.06319/MACS1149-new.bbl.
syntax error in line 18: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.05714-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.05714) | **UNCOVER spectroscopy confirms a surprising ubiquity of AGN in red  galaxies at $z>5$**  |
|| J. E. Greene, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2023-09-13*|
|*Comments*| *23 pages, 9 figures, 5 tables, submitted to ApJ*|
|**Abstract**| JWST is revealing a new population of dust-reddened broad-line active galactic nuclei (AGN) at redshifts $z\gtrsim5$. Here we present deep NIRSpec/Prism spectroscopy from the Cycle 1 Treasury program UNCOVER of 15 AGN candidates selected to be compact, with red continua in the rest-frame optical but with blue slopes in the UV. From NIRCam photometry alone, they could have been dominated by dusty star formation or AGN. Here we show that the majority of the compact red sources in UNCOVER are dust-reddened AGN: $60\%$ show definitive evidence for broad-line H$\alpha$ with FWHM$\, >2000$ km/s, for $20\%$ current data are inconclusive, and $20\%$ are brown dwarf stars. We propose an updated photometric criterion to select red $z>5$ AGN that excludes brown dwarfs and is expected to yield $>80\%$ AGN. Remarkably, among all $z_{\rm phot}>5$ galaxies with F277W$-$F444W$>1$ in UNCOVER at least $33\%$ are AGN regardless of compactness, climbing to at least $80\%$ AGN for sources with F277W$-$F444W$>1.6$. The confirmed AGN have black hole masses of $10^7-10^9$ M$_{\odot}$. While their UV-luminosities ($-16>M_{\rm UV}>-20$ AB mag) are low compared to UV-selected AGN at these epochs, consistent with percent-level scattered AGN light or low levels of unobscured star formation, the inferred bolometric luminosities are typical of $10^7-10^9$ M$_{\odot}$ black holes radiating at $\sim 10-40\%$ of Eddington. The number densities are surprisingly high at $\sim10^{-5}$ Mpc$^{-3}$ mag$^{-1}$, 100 times more common than the faintest UV-selected quasars, while accounting for $\sim1\%$ of the UV-selected galaxies. While their UV-faintness suggest they may not contribute strongly to reionization, their ubiquity poses challenges to models of black hole growth. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.06311-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.06311) | **Spatially-resolved Spectroscopic Analysis of Ly$α$ Haloes: Radial  Evolution of the Ly$α$ Line Profile out to 60 kpc**  |
|| Y. Guo, et al. -- incl., <mark>L. Boogaard</mark> |
|*Appeared on*| *2023-09-13*|
|*Comments*| *Submitted to A&A Letters*|
|**Abstract**| Deep MUSE observations have unveiled pervasive Ly$\alpha$ haloes (LAHs) surrounding high-redshift star-forming galaxies. However, the origin of the extended Ly$\alpha$ emission is still a subject of debate. We analyse the average spatial extent and spectral variation of the circumgalactic LAHs by stacking a sample of 155 Ly$\alpha$ emitters (LAEs) at redshift $3<z<4$ in the MUSE Extremely Deep Field (MXDF). With respect to the Ly$\alpha$ red peak of the target LAE, the Ly$\alpha$ line peak becomes increasingly more blueshifted out to a projected distance of at least 60 kpc, where the velocity offset is $\approx$ 250 km/s. This signal is observed in both the mean and median stacks, and is thus a generic property of the LAE sample with typical Ly$\alpha$ luminosity $\mathrm{\approx 10^{41.1} erg\,s^{-1}}$. We discuss multiple scenarios to explain the blueshift of the circumgalactic Ly$\alpha$ line. The most plausible one is a combination of outflows and inflows. In the inner region of the LAH, the Ly$\alpha$ photons are produced by the central star formation and then scattered within outflows. At larger radii, the infalling cool gas shapes the observed Ly$\alpha$ blueshift. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.06319-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.06319) | **Spatially-resolved H$α$ and ionizing photon production efficiency  in the lensed galaxy MACS1149-JD1 at a redshift of 9.11**  |
|| J. Álvarez-Márquez, et al. -- incl., <mark>S. Bosman</mark>, <mark>L. Boogaard</mark>, <mark>F. Walter</mark> |
|*Appeared on*| *2023-09-13*|
|*Comments*| *11 pages, 6 figures, submitted to A&A*|
|**Abstract**| We present MIRI/JWST medium resolution spectroscopy (MRS) and imaging (MIRIM) of the lensed galaxy MACS1149-JD1 at a redshift of $z$=9.1092$\pm$0.0002 (Universe age about 530 Myr). We detect, for the first time, spatially-resolved H$\alpha$ emission in a galaxy at redshift above 9. The structure of the H$\alpha$ emitting gas consists of two clumps, S and N. The total H$\alpha$ luminosity implies an instantaneous star formation rate of 5.3$\pm$0.4 $M_{\odot}$ yr$^{-1}$ for solar metallicities. The ionizing photon production efficiency, $\log(\zeta_\mathrm{ion})$, shows a spatially-resolved structure with values of 25.55$\pm$0.03, 25.47$\pm$0.03, and 25.91$\pm$0.09 Hz erg$^{-1}$ for the integrated galaxy, and clumps S and N, respectively. The H$\alpha$ rest-frame equivalent width, EW$_{0}$(H$\alpha$), is 491$^{+334}_{-128}$ \'Angstrom for the integrated galaxy, but presents extreme values of 363$^{+187}_{-87}$ \'Angstrom and $\geq$1543 \'Angstrom for clumps S and N, respectively. The spatially-resolved ionizing photon production efficiency is within the range of values measured in galaxies at redshift above six, and well above the canonical value (25.2$\pm$0.1 Hz erg$^{-1}$). The extreme difference of EW$_{0}$(H$\alpha$) for Clumps S and N indicates the presence of a recent (few Myr old) burst in clump N and a star formation over a larger period of time (e.g. 100$-$200 Myr) in clump S. Finally, clump S and N show very different H$\alpha$ kinematics with velocity dispersions of 56$\pm$4 km s$^{-1}$ and 113$\pm$33 km s$^{-1}$, likely indicating the presence of outflows or increased turbulence in the clump N. The dynamical mass, $M_\mathrm{dyn}$= (2.4$\pm$0.5)$\times$10$^{9}$ $M_{\odot}$, is within the range previously measured with the spatially-resolved [OIII]88$\mu$m line. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.06235-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.06235) | **AugerPrime Surface Detector Electronics**  |
|| P. A. Collaboration, et al. |
|*Appeared on*| *2023-09-13*|
|*Comments*| **|
|**Abstract**| Operating since 2004, the Pierre Auger Observatory has led to major advances in our 8 understanding of the ultra-high-energy cosmic rays. The latest findings have revealed new insights 9 that led to the upgrade of the Observatory, with the primary goal of obtaining information on the 10 primary mass of the most energetic cosmic rays on a shower-by-shower basis. In the framework of the 11 upgrade, called AugerPrime, the 1660 water-Cherenkov detectors of the surface array are equipped 12 with plastic scintillators and radio antennas, allowing us to enhance the composition sensitivity. 13 To accommodate new detectors and to increase experimental capabilities, the electronics is also 14 upgraded. This includes better timing with up-to-date GPS receivers, higher sampling frequency, 15 increased dynamic range, and more powerful local processing of the data. In this paper, the design 16 characteristics of the new electronics and the enhanced dynamic range will be described. The 17 manufacturing and test processes will be outlined and the test results will be discussed. The 18 calibration of the SD detector and various performance parameters obtained from the analysis of 19 the first commissioning data will also be presented. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2309.05714.md
    + _build/html/tmp_2309.05714/./select.png
    + _build/html/tmp_2309.05714/./target_color.png
    + _build/html/tmp_2309.05714/./zspec_zphot.png
    + _build/html/tmp_2309.05714/./model_4286.png
exported in  _build/html/2309.06311.md
    + _build/html/tmp_2309.06311/figures/fig_stack_lya_profile_z1.png
    + _build/html/tmp_2309.06311/figures/fig_stack_lya_profile_zsys.png
    + _build/html/tmp_2309.06311/figures/fig_Lya_NBs.png
exported in  _build/html/2309.06319.md
    + _build/html/tmp_2309.06319/./HST_MIRIM_HaLineMap_clumps.png
    + _build/html/tmp_2309.06319/./Photon_efficiency_redshift.png
    + _build/html/tmp_2309.06319/./EW_Halpha_evo_Javi.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\url}[1]{\href{#1}{#1}}$
$\newcommand{\dodoi}[1]{doi:~\href{http://doi.org/#1}{\nolinkurl{#1}}}$
$\newcommand{\doeprint}[1]{\href{http://ascl.net/#1}{\nolinkurl{http://ascl.net/#1}}}$
$\newcommand{\doarXiv}[1]{\href{https://arxiv.org/abs/#1}{\nolinkurl{https://arxiv.org/abs/#1}}}$
$\newcommand{\angstrom}{\text{\normalfontÅ}}$
$\newcommand{\deltare}{\mathrm{\Delta log(r_e)}}$
$\newcommand{\oiii}{[\ion{O}{3}]}$
$\newcommand{\nii}{[\ion{N}{2}]}$
$\newcommand{\nev}{[\ion{Ne}{5}]}$
$\newcommand{\msun}{M_{\odot}}$
$\newcommand{\sersic}{Sérsic}$
$\newcommand{\mkgroup}{M_{\rm K,  group}}$
$\newcommand{\mstar}{M_*}$
$\newcommand{\mbh}{M_{\rm BH}}$
$\newcommand{\lbol}{L_{\rm bol}}$
$\newcommand{\jwst}{\ensuremath{JWST}}$
$\newcommand{\ivo}[1]{\textcolor{violet}{#1}}$
$\newcommand{\catnr}{50,000}$
$\newcommand{\bronzenr}{40}$
$\newcommand{\silvernr}{26}$
$\newcommand{\goldnr}{17}$
$\newcommand{\sourcenum}{17}$
$\newcommand{\}{natexlab}$</div>



<div id="title">

# UNCOVER spectroscopy confirms a surprising ubiquity of AGN in red galaxies at $z>5$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2309.05714-b31b1b.svg)](https://arxiv.org/abs/2309.05714)<mark>Appeared on: 2023-09-13</mark> -  _23 pages, 9 figures, 5 tables, submitted to ApJ_

</div>
<div id="authors">

J. E. Greene, et al. -- incl., <mark>A. d. Graaff</mark>

</div>
<div id="abstract">

**Abstract:** $\jwst$ is revealing a new population of dust-reddened broad-line active galactic nuclei (AGN) at redshifts $z\gtrsim5$ . Here we present deep NIRSpec/Prism spectroscopy from the Cycle 1 Treasury program UNCOVER of 17 AGN candidates selected to be compact, with red continua in the rest-frame optical but with blue slopes in the UV. From NIRCam photometry alone, they could have been dominated by dusty star formation or AGN. Here we show that the majority of the compact red sources in UNCOVER are dust-reddened AGN: 60 \% show definitive evidence for broad-line H $\alpha$ with FWHM $>2000$ km/s, for 20 \% current data are inconclusive, and $20\%$ are brown dwarf stars. We propose an updated photometric criterion to select red $z>5$ AGN that excludes brown dwarfs and is expected to yield $>80\%$ AGN. Remarkably, among all $z_{\rm phot}>5$ galaxies with F277W $-$ F444W $>1$ in UNCOVER at least $33\%$ are AGN regardless of compactness, climbing to at least $80$ \% AGN for sources with F277W $-$ F444W $>1.6$ .The confirmed AGN have black hole masses of $10^7-10^9$  $\msun$ . While their UV-luminosities ( $-16>M_{\rm UV}>-20$ AB mag) are low compared to UV-selected AGN at these epochs, consistent with percent-level scattered AGN light or low levels of unobscured star formation, the inferred bolometric luminosities are typical of $10^7-10^9$  $\msun$ black holes radiating at $\sim 10-40\%$ of Eddington. The number densities are surprisingly high at $\sim10^{-5}$ Mpc $^{-3}$ mag $^{-1}$ , 100 times more common than the faintest UV-selected quasars, while accounting for $\sim1\%$ of the UV-selected galaxies. While their UV-faintness suggest they may not contribute strongly to reionization, their ubiquity poses challenges to models of black hole growth.

</div>

<div id="div_fig1">

<img src="tmp_2309.05714/./select.png" alt="Fig5" width="100%"/>

**Figure 5. -** AGN in the context of high-redshift red galaxies. The grey points are the UNCOVER catalog for F444W$>27.7$ mag and F444W S/N$>14$. Top left: NIRCam F115W$-$F200W versus F277W$-$F444W bi-color selection identifies  galaxies like those in \citet{Labbe:2023} that have blue rest-frame UV continuum and red rest-frame optical continuum ("v-shape" SED). The AGN candidate sample of \citet{Labbe:2023uncover} is a subset of that ($\sim50\%$), selected to also have red optical sloped continuum via a cut in two adjacent filters (top right, and compact size (bottom left), using a ratio of aperture fluxes as a proxy for size. Brown dwarf star contaminants generally have bluer F115W$-$F200W than galaxies and can therefore be isolated. Overplot are synthetic color tracks from the LOWZ brown dwarf atmosphere models \citep{2021ApJ...915..120M} for T$_{\rm eff}$$\leq$ 1600 K and solar [M/H] = 0 and $-1.5$. The $v-shape$ criterion alone is very effective at selecting for $z>5$ galaxies. A selection including the compact + red criterion is efficient at selecting red AGN.
 (*fig:newsample*)

</div>
<div id="div_fig2">

<img src="tmp_2309.05714/./target_color.png" alt="Fig2.1" width="50%"/><img src="tmp_2309.05714/./zspec_zphot.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** Left: The primary color-color selection used to select the compact red sources. We show the full sample of 40 compact red sources (open circles), the 17 clean targets (yellow) and the spectroscopically targeted sources (black dots are those observed with the MSA). Comparison of the photometric redshifts measured from the NIRCam photometry using custom templates from L23, as compared with the spectroscopic redshifts. The brown dwarfs are indicated with stars.
 (*fig:target*)

</div>
<div id="div_fig3">

<img src="tmp_2309.05714/./model_4286.png" alt="Fig6" width="100%"/>

**Figure 6. -** We illustrate our preferred model for the particular red and UV slopes seen in our objects using MSAID4286. The intrinsic AGN continuum (red) is highly reddened. Thus, the UV component cannot be explained by the primary AGN continuum. Here, we explore the possibility that the UV comes from scattered light, at 2.5\% of the intrinsic UV (shown schematically in light blue). For illustration, we show here the \citet{Temple:2021} template. However, we achieve even better fits when we use the observed UV slope as the intrinsic power-law AGN shape. We also overplot a stellar-population fit to the UV-side of the spectrum with \texttt{Bagpipes}\citep{Carnall:2018, Carnall:2019}, again to illustrate that with moderate star-formation rates of a few solar masses per year, and $A_V \sim 0.6$ mag, it is possible to fit the UV continuum slope with star light as well.
 (*fig:uvfit*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2309.05714"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\lya}{Ly\alpha}$
$\newcommand{\hi}{H~{\small I}}$
$\newcommand{\mgi}{Mg~{\small I}}$
$\newcommand{\mgii}{Mg~{\small II}}$
$\newcommand{\mgiia}{\mgii~\lambda2796}$
$\newcommand{\mgiib}{\mgii~\lambda2803}$
$\newcommand{\mgiiab}{\mgii~\lambda\lambda2796,2803}$
$\newcommand{\civ}{C~{\small IV}}$
$\newcommand{\heii}{He~{\small II}}$
$\newcommand{\ciii}{C~{\small III}]}$
$\newcommand{\oii}{[O~{\small II}]}$</div>



<div id="title">

# Spatially-resolved Spectroscopic Analysis of $\lya$ Haloes

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2309.06311-b31b1b.svg)](https://arxiv.org/abs/2309.06311)<mark>Appeared on: 2023-09-13</mark> -  _Submitted to A&A Letters_

</div>
<div id="authors">

Y. Guo, et al. -- incl., <mark>L. Boogaard</mark>

</div>
<div id="abstract">

**Abstract:** Deep MUSE observations have unveiled pervasive $\lya$ haloes (LAHs) surrounding high-redshift star-forming galaxies. However, the origin of the extended $\lya$ emission is still a subject of debate.We analyse the average spatial extent and spectral variation of the circumgalactic LAHs by stacking a sample of 155 $\lya$ emitters (LAEs) at redshift $3<z<4$ in the MUSE Extremely Deep Field (MXDF).With respect to the $\lya$ red peak of the target LAE, the $\lya$ line peak becomes increasingly more blueshifted out to a projected distance of at least 60 kpc, where the velocity offset is $\approx$ 250 km/s.This signal is observed in both the mean and median stacks, and is thus a generic property of the LAE sample with typical $\lya$ luminosity $\mathrm{\approx 10^{41.1} erg s^{-1}}$ .We discuss multiple scenarios to explain the blueshift of the circumgalactic $\lya$ line.The most plausible one is a combination of outflows and inflows. In the inner region of the LAH, the $\lya$ photons are produced by the central star formation and then scattered within outflows. At larger radii, the infalling cool gas shapes the observed $\lya$ blueshift.

</div>

<div id="div_fig1">

<img src="tmp_2309.06311/figures/fig_stack_lya_profile_z1.png" alt="Fig1" width="100%"/>

**Figure 1. -** $\lya$ line profiles at different distances from the central galaxy.
The spectra in the left and right columns are derived from the mean- and median-stacked datacubes.
Different rows show different radial bins ranging from 1" (7 kpc) to 8" (59 kpc).
The vertical dashed lines denote the peak of the central $\lya$ line. The horizontal grey shaded regions show the 1$\sigma$ error range.
 (*fig_Lya_profile*)

</div>
<div id="div_fig2">

<img src="tmp_2309.06311/figures/fig_stack_lya_profile_zsys.png" alt="Fig2" width="100%"/>

**Figure 2. -** 
As Figure \ref{fig_Lya_profile}, except that here all the mini-datacubes are re-aligned by the estimated $z_{sys}$ instead of $z_{Ly\alpha}$.
 (*fig_Lya_profile_zsys*)

</div>
<div id="div_fig3">

<img src="tmp_2309.06311/figures/fig_Lya_NBs.png" alt="Fig3" width="100%"/>

**Figure 3. -** Surface brightness maps of the median-stacked $\lya$ emission.
Each panel shows the $\lya$ pseudo-NB in a velocity interval of 200 km/s. The zero point corresponds to the peak of the $\lya$ line.
The NBs have been smoothed using a Gaussian kernel of width of $2.3"$.
The grey contours correspond to $\lya$ significance levels of 2, 4 and 6 $\sigma$(dotted, dashed and solid, respectively).
 (*fig_Lya_NBs*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2309.06311"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\arcs}{\arcsec\xspace}$
$\newcommand{\CII}{[\ion{C}{II}]158\mum}$
$\newcommand{\oddpm}[2]{\raisebox{0.5ex}{\tiny\substack{+#1 \-#2}}}$</div>



<div id="title">

# Spatially-resolved H$\alpha$ and ionizing photon production efficiency in the lensed galaxy MACS1149-JD1 at a redshift of 9.11

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2309.06319-b31b1b.svg)](https://arxiv.org/abs/2309.06319)<mark>Appeared on: 2023-09-13</mark> -  _11 pages, 6 figures, submitted to A&A_

</div>
<div id="authors">

J. Álvarez-Márquez, et al. -- incl., <mark>S. Bosman</mark>, <mark>L. Boogaard</mark>, <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** We present MIRI/JWST medium resolution spectroscopy (MRS) and imaging (MIRIM) of the lensed galaxy MACS1149-JD1 at a redshift of $z$ = 9.1092 $\pm$ 0.0002, when the Universe was about 530 Myr old. We detect, for the first time, spatially-resolved H $\alpha$ emission in a galaxy at redshift above 9. The structure of the H $\alpha$ emitting gas consists of two clumps, S and N, carrying about $60\%$ and $40\%$ of the total flux, respectively. The total H $\alpha$ luminosity implies an instantaneous star formation rate in the range of 3.2 $\pm$ 0.3 and 5.3 $\pm$ 0.4 $M_{\odot}$ yr $^{-1}$ for sub-solar and solar metallicities. The ionizing photon production efficiency, $\log(\zeta_\mathrm{ion})$ , shows a spatially-resolved structure with values of 25.55 $\pm$ 0.03, 25.47 $\pm$ 0.03, and 25.91 $\pm$ 0.09 Hz erg $^{-1}$ for the integrated galaxy, and clumps S and N, respectively. The H $\alpha$ rest-frame equivalent width, EW $_{0}$ (H $\alpha$ ), is 491 $^{+334}_{-128}$ $Å$ for the integrated galaxy, but presents extreme values of 363 $^{+187}_{-87}Å$ and $\geq$ 1543 $Å$ for clumps S and N, respectively. The spatially-resolved ionizing photon production efficiency is within the range of values measured in galaxies at redshift above six, and well above the canonical value (25.2 $\pm$ 0.1 Hz erg $^{-1}$ ). The EW $_{0}$ (H $\alpha$ ) is a factor 3-4 lower than the predicted value at $z$ = 9.11 based on the extrapolation of the evolution of the EW $_{0}$ (H $\alpha$ ) with redshifts, $\propto$ (1+z) $^{2.1}$ , including galaxies detected with JWST. The extreme difference of EW $_{0}$ (H $\alpha$ ) for Clumps S and N indicates the presence of a recent (few Myr old) burst in clump N and a star formation over a larger period of time (e.g. 100 $-$ 200 Myr) in clump S, placing the initial formation of MACS1149-JD1 at a $z$ $\sim$ 11-12. The different ages of the stellar population place MACS1149-JD1 and clumps N and S at different locations in the log( $\zeta_\mathrm{ion}$ ) to EW $_{0}$ (H $\alpha$ ) plane and above the main relation defined from intermediate and high redshift (z=3-7) galaxies detected with JWST. Finally, clump S and N show very different H $\alpha$ kinematics with velocity dispersions of 56 $\pm$ 4 km s $^{-1}$ and 113 $\pm$ 33 km s $^{-1}$ , likely indicating the presence of outflows or increased turbulence in the clump N. The dynamical mass $M_\mathrm{dyn}$ = (2.4 $\pm$ 0.5) $\times$ 10 $^{9}$ $M_{\odot}$ , obtained from the size of the integrated H $\alpha$ ionized nebulae and its velocity dispersion, is within the range previously measured with the spatially-resolved [ OIII ] 88 $\mu$ m line.

</div>

<div id="div_fig1">

<img src="tmp_2309.06319/./HST_MIRIM_HaLineMap_clumps.png" alt="Fig5" width="100%"/>

**Figure 5. -** Images of MACS1149-JD1 presenting the observed stellar and ionized gas structure. From left to right: archival HST WFPC3/F160W from Hubble Advanced Product Multi-Visit Mosaic (HAP-MVM) program, MIRIM F560W image and MRS H$\alpha$ line map. The H$\alpha$ line map is generated by integrating H$\alpha$ line emission in the velocity range, -150 < $v$[km s$^{-1}$] < 150. The origin of the image corresponds to (RA [deg], DEC [deg]) of (177.389945, +22.412722). The black dashed ellipse indicates the aperture used to perform the MIRIM F560W photometry and the MRS 1D spectral extraction. Dotted black ellipses identify the H$\alpha$ emitting regions named as clump N and S, and spatially coincident with the two emitting regions in the HST F160W image. White area represents the spatial resolution (PSF FWHM) of each observation. MACS1149-JD1 shows an elongated structure due to the lens magnification of cluster MACS J11491+2223 \citep{Zheng+12}.  (*fig:HaLineMap*)

</div>
<div id="div_fig2">

<img src="tmp_2309.06319/./Photon_efficiency_redshift.png" alt="Fig1" width="100%"/>

**Figure 1. -** Ionizing photon production efficiency as a function of redshift. MACS1149-JD1 is represented by circles, where we distinguish between the values of the integrated galaxy (red), and the spatially-resolved clumps N (green) and S (yellow). Squares: galaxies spectroscopically identified at $z$$\gtrsim$ 6 with JWST \citep{Fujimoto+23,Jung+23,Morishita+23,Saxena+23,Matthee+23,Tang+23,Atek+23}. Green area: canonical value for the ionizing photon efficiency \citep{Robertson+13}. Black line and gray area: variation of the photon efficiency with redshift and its uncertainty \citep{Matthee+17}. (*fig:photon-z*)

</div>
<div id="div_fig3">

<img src="tmp_2309.06319/./EW_Halpha_evo_Javi.png" alt="Fig2" width="100%"/>

**Figure 2. -** Evolution of the rest-frame equivalent width of H$\alpha$ as a function of redshift. MACS1149-JD1 is represented by circles, where we distinguish between the values of the integrated galaxy (red), and the spatially-resolved clumps N (green) and S (yellow). Additional data includes samples detected with JWST at redshifts of around 6 \citep{Sun+22,Ning+23,Prieto-Lyon+23} and 7$-$8 \citep{Rinaldi+23}, as well as pre-JWST galaxies at redshifts 0.5 to 8. The pre-JWST value at redshift 8 corresponds to the median stacking of a sample of 102 Lyman-break galaxies \citep{Stefanon+22}. The dashed, dark red line represents the best fit to all data points, including the MIDIS sources, and described by a single law EW$_{0}$(H$\alpha$) $\propto$(1+$z$)$^{2.1}$ by \cite{Rinaldi+23}. The dotted broken line represents previous fits with pre-JWST data up to a redshift of six \citep{Faisst+16}.
       (*fig:EW*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2309.06319"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

269  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
